# Analysis market transactions (positions)

### Installing all needed libraries

In [1]:
import pathlib
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from helpers import table_with_filter, tables_show, range_slider

### Reading all sources files for analysis

In [2]:
SOURCE_PATH = pathlib.Path("./sources/")

pnl_df = pd.read_excel(SOURCE_PATH / "PnL.xlsx")
positions_detail_df = pd.read_excel(SOURCE_PATH / "position_details.xlsx")
instrument_detail_df = pd.read_json(SOURCE_PATH / "instrument_details.json").T

### Preparing and Denormalization data into one result Table 

In [3]:
instrument_detail_df.reset_index(inplace=True, names=["Instrument_Id"])
positions_all_view = pd.merge(positions_detail_df, instrument_detail_df, on="Instrument_Id")
positions_all_view = pd.merge(positions_all_view, pnl_df, on="Position_Id")

### Manipulating aggregating and calculating

#### Tables those show the positions and instrument and their DELTA & GAMMA

In [4]:
tables_show(
    [
        positions_all_view.groupby(by=["Position_Id"]).agg({"DELTA":"mean", "GAMMA":"mean"}),
        positions_all_view.groupby(by=["Position_Id","Instrument_Id","Type"]).agg({"DELTA":"mean", "GAMMA":"mean"}),
    ],
    titles=["Positions and their DELTA & GAMMA", "Positions by Instrument and their DELTA & GAMMA"]
)

#### Table that shows DELTA by each sector

In [5]:
tables_show([positions_all_view.groupby(by="Sector").agg({"DELTA": 'mean'})], titles=["DELTA by each Sector"])

#### Table that shows the positions and instrument and their DELTA & GAMMA average values and filtering by Type Instrument

In [6]:
table_with_filter(
    positions_all_view.groupby(by=["Position_Id","Instrument_Id","Type"]).agg({"DELTA":"mean", "GAMMA":"mean"}),
    title="Positions and their DELTA and GAMMA with filtering by Instrument Type"
)

#### Top 10 positions with positive and negative market risk by DELTA

In [7]:
TOP_RECORDS = 10
aggregate_data = positions_all_view.groupby(by=["Position_Id"]).agg({"Quantity":"sum", "DELTA":"mean"})
tables_show(
    dfs=[
            aggregate_data.sort_values(by='DELTA', ascending=False).iloc[:TOP_RECORDS],
            aggregate_data.sort_values(by='DELTA', ascending=True).iloc[:TOP_RECORDS],
    ],
    titles=["TOP 10 Most Positive DELTA", "TOP 10 Most Negative DELTA"]
)

#### Total PnL per each Region with Filtering by PriceChange and VolatilityChange

In [9]:
# Filter sliders by PriceChange and VolatilityChange
price_slider = range_slider('Price Change Filter:')
volatility_slider = range_slider('Volatility Change Filter:')

# Function update table and plot by filter params
def show_table_with_plot(price_range, volatility_range):
    filtered_df = positions_all_view[
        (positions_all_view["PriceChange"] >= price_range[0])
        & (positions_all_view["PriceChange"] <= price_range[1])
        & (positions_all_view["VolatilityChange"] >= volatility_range[0])
        & (positions_all_view["VolatilityChange"] <= volatility_range[1])
    ]
    total_pnl_per_region = filtered_df.groupby("Region")["PnL"].sum()
    fig = make_subplots(
        rows=2,
        cols=1,
        specs=[[{"type": "table"}], [{"type": "bar"}]],
    )
    fig.add_trace(
        go.Table(
            header=dict(values=["Region", "PnL"]),
            cells=dict(values=[total_pnl_per_region.index, total_pnl_per_region.values]),
        ),
        row=1,
        col=1,
    )
    fig.add_trace(go.Bar(x=total_pnl_per_region.index, y=total_pnl_per_region.values))
    fig.update_layout(title_text="Total PnL per each Region")
    fig.show()


# create interactive table
widgets.interact(show_table_with_plot, price_range=price_slider, volatility_range=volatility_slider)


interactive(children=(IntRangeSlider(value=(-25, 10), continuous_update=False, description='Price Change Filte…

<function __main__.show_table_with_plot(price_range, volatility_range)>